# Les Optimisations de mix de produits ( produits mix )

## Introduction.


Les optimisations de type mix de produits sont les premières abordées en optimisation.

Elles permettent d'optimiser la production et le rendement commercial d'une usine ou d'une organisation agricole, et d'éviter en premier ressort des logiques de production aberrantes.

Elles permettent à une usine de savoir quels produits il vaut mieux produire dans son éventail de produits, afin de maximiser son profit.


Un exemple d'optimisation mixe de produits pour une organisation agricole 
dans mon application www.solvgraph.com :
<div style="text-align:center">
<img src="img/product-mix1.png">
</div>

Dans ses versions non linéaires, elles permettent plus de sophistication et permettent de coller à la réalité.

Dans cette étude, je vais utiliser plusieurs solveurs, et faire des optimisations linéaires et non 
linéaires.

Un exemple d'optimisation dans mon application www.solvgraph.com :
<div style="text-align:center">
<img src="img/product_mixe2.jpg">
</div>


Etude globale proposée par <b>Estelle Derrien - Github15000</b>

CREATION EN COURS


# Sommaire 

- 1. Maximisation de profit linéaire.
        - Notre problème de base
        - Modélisation mathématique
        - Résolution avec Python Pulp
- 3. A quoi sert l'analyse de sensitivité ?
- 4. Ajout des coûts fixes et variables.
- 5. Forcer la production d'un élément.
- 6. Ajout de contraintes binaires/conditionnelles.
- 7. Ajout de Multiples de.
- 8. Prise en compte des Invendus.
- 9. Ajout de contraintes Piecewize.
    - Description
    - Résolution avec Cplex
    - Résolution avec Python Pulp
- 10. Maximisation de profit non linéaire - Production de 3 objets.
    - Notre problème de base
    - Modélisation mathématique
    - Exemple
    - Exemple avec boucles For
    - Résolution avec Python Gekko
        - Sans Boucle For
        - Avec Boucle For
    - Résolution avec Cplex
    - Ajout de contraintes binaires
    - Ajout de contraintes piecewize




# Maximisation de profit linéaire - Production de 3 objets.


# Notre problème de base.

- J'ai le choix entre 3 types de vaches : Charolaises, Blanc Bleuc et salers.

- J'ai 600 kgs de nourriture, 700 litres d'eau en stock .

- Chaque vache consomme respectivement 7kgs,7kgs,8kgs de nourriture par jour.

- Chaque vache consomme respectivement 4.5l,9l et 3l d'eau par jour.

- Chaque vache rapporte respectivement 100,110,et 115 euros.

Quelles vaches faut il produire afin de maximiser mon profit ?



# Modélisation Mathématique.

- Soit V un ensemble de vaches indicées par i (v1,v2,v3...vi)
- Soit Nij la nourriture consommée par chaque vache, c'est une matrice ( ou un tableau).
- Soit Eij l'eau consommée par chaque vache, c'est une matrice ( ou un tableau).
- Soit Pij le profit rapporté par chaque vache , c'est une matrice ( ou un tableau).

je veux maximiser mon profit.


# Résolution avec Python Pulp

Comme on a que 3 vaches, on peut résoudre sans utiliser les boucles FOR, qui paraissent un peu compliquées quand on débute.

In [36]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  
# Créer les variables du problème 
charolaise = p.LpVariable("charolaise", 0, None, p.LpInteger)   
blanc_bleu = p.LpVariable("blanc_bleu", 0, None, p.LpInteger)   
salers = p.LpVariable("salers", 0, None, p.LpInteger)   
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Lp_prob +=  100 * charolaise + 110 * blanc_bleu + 115 * salers
# Les contraintes : 
# nourriture
Lp_prob += 7 * charolaise + 7 * blanc_bleu + 8 * salers <= 600
# eau
Lp_prob += 4.5* charolaise + 9 * blanc_bleu + 3 * salers <= 700
# Résoudre
Lp_prob.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Lp_prob.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", p.value(Lp_prob.objective))

blanc_bleu = 72.0
charolaise = 0.0
salers = 12.0
Profit total maximisé =  9300.0


# Ajout des coûts fixes et variables.

Mon exploitation a un coût fixe de 1200 euros.
Chaque vache implique un coût variable de respectivement 10,12,9.5 euros.

Je veux inclure ces éléments à la fonction objectif afin qu'elle les prenne en compte lors de l'optimisation.

Note:  On est toujours dans le cadre d'un problème linéaire, les coûts variables des vaches est donc proportionnel au nombre de vache, chaque vache a un coût fixe directement appliqué, il ne diminue pas avec le nombre, si c'était le cas, on serait dans un problème non linéaire.

In [37]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
# Nos coûts
cout_fixe = 1000
cout_charolaise = 10
cout_blanc_bleu = 12
cout_salers = 9.5
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  
# Créer les variables du problème 
charolaise = p.LpVariable("charolaise", 0, None, p.LpInteger)   
blanc_bleu = p.LpVariable("blanc_bleu", 0, None, p.LpInteger)   
salers = p.LpVariable("salers", 0, None, p.LpInteger)   
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Lp_prob +=  ((100 - cout_charolaise)  * charolaise + (110 - cout_blanc_bleu) * blanc_bleu + (115 - cout_salers) * salers ) - cout_fixe
# Les contraintes : 
# nourriture
Lp_prob += 7 * charolaise + 7 * blanc_bleu + 8 * salers <= 600
# eau
Lp_prob += 4.5* charolaise + 9 * blanc_bleu + 3 * salers <= 700
# Résoudre
Lp_prob.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Lp_prob.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", p.value(Lp_prob.objective))

blanc_bleu = 72.0
charolaise = 0.0
salers = 12.0
Profit total maximisé =  7322.0


# Forcer la production d'un élément.

On voit dans notre résultat que le solveur nous dit de ne pas produire de charolaises, afin de maximiser notre profit.
Mais je veux tout de même avoir 20 Charolaises au minimum dans mon cheptel et 20 salers au minimum, même si cela implique une perte de profit.


In [38]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
# Nos coûts
cout_fixe = 1000
cout_charolaise = 10
cout_blanc_bleu = 12
cout_salers = 9.5
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  
# Créer les variables du problème 
charolaise = p.LpVariable("charolaise", 0, None, p.LpInteger)   
blanc_bleu = p.LpVariable("blanc_bleu", 0, None, p.LpInteger)   
salers = p.LpVariable("salers", 0, None, p.LpInteger)   
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Lp_prob +=  ((100 - cout_charolaise)  * charolaise + (110 - cout_blanc_bleu) * blanc_bleu + (115 - cout_salers) * salers ) - cout_fixe
# Les contraintes : 
# nourriture
Lp_prob += 7 * charolaise + 7 * blanc_bleu + 8 * salers <= 600
# eau
Lp_prob += 4.5* charolaise + 9 * blanc_bleu + 3 * salers <= 700
# Minimum de 20 charolaises
Lp_prob += charolaise >= 20
# Minimum de 20 salers
Lp_prob += salers >= 20
# Résoudre
Lp_prob.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Lp_prob.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", p.value(Lp_prob.objective))

blanc_bleu = 36.0
charolaise = 20.0
salers = 26.0
Profit total maximisé =  7071.0
